In [2]:
import requests
import json
import base64
import tensorflow as tf

inputs = {
    "Age": 12,
    "BMI": 0.1848072111606598,
    "BloodPressure": 12,
    "DiabetesPedigreeFunction": 0.4571339786052704,
    "Glucose": 12,
    "Insulin": 8,
    "Pregnancies": 5,
    "SkinThickness": 12
}

def float_feature(value):
    return tf.train.Feature(
        float_list=tf.train.FloatList(
            value=[value]
        ),
    )

def int_feature(value):
    return tf.train.Feature(
        int64_list=tf.train.Int64List(
            value=[value]
        ),
    )

def prepare_json(inputs: dict):
    data = dict()
    
    for keys, values in inputs.items():
        if isinstance(values, int):
            data[keys] = int_feature(values)
        elif isinstance(values, float):
            data[keys] = float_feature(values)
            
    example = tf.train.Example(
        features=tf.train.Features(feature=data)
    ).SerializeToString()
    
    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]

    return json.dumps({
        "signature_name": "serving_default",
        "instances": result,
    })

def make_predictions(inputs):
    json_data = prepare_json(inputs)
    
    endpoint = "https://mlpipeline-production.up.railway.app/v1/models/cc-model:predict"
    response = requests.post(endpoint, data=json_data)
    prediction = response.json()

    prediction = response.json()["predictions"][0][0]
    
    if prediction < 0.6:
        return "Not diagnosed diabet"
    else:
        return "diagnosed diabet"

make_predictions(inputs)

'Not diagnosed diabet'